In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, FloatType
from pathlib import Path

qualy_bronze_path = Path.cwd() / 'bronze' / 'qualifying'
qualy_silver_path = Path.cwd() / 'silver' / 'qualifying'

spark = SparkSession.builder.appName('f1Practice').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 12:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/16 12:54:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
qualy_schema = StructType(fields=[
    StructField('constructorId', IntegerType(), True),
    StructField('driverId', IntegerType(), True),
    StructField('position', IntegerType(), True),
    StructField('q1', StringType(), True),
    StructField('q2', StringType(), True),
    StructField('q3', StringType(), True),
    StructField('qualifyId', IntegerType(), True),
    StructField('raceId', IntegerType(), True)
])

df = spark.read.schema(qualy_schema).option('multiline', True).json(str(qualy_bronze_path))

In [8]:
final_df = df.withColumnRenamed('constructorId', 'constructor_id') \
            .withColumnRenamed('driverId', 'driver_id') \
            .withColumnRenamed('qualifyId', 'qualify_id') \
            .withColumnRenamed('raceId', 'race_id') \
            .withColumn('ingestion_date', current_timestamp())

In [9]:
final_df.write.mode('overwrite').parquet(str(qualy_silver_path))